In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
pd.io.sql.read_sql('PRAGMA table_info(test);', con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [4]:
pd.io.sql.read_sql('SELECT * FROM test LIMIT 10;', con)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [5]:
pd.io.sql.read_sql('SELECT * FROM deadlines', con, index_col='index', 
                  parse_dates=['deadlines'])

,labs,deadlines
index,,
0,laba04,2020-04-26 23:59:59
1,laba04s,2020-04-26 23:59:59
2,laba05,2020-05-03 23:59:59
4,laba06,2020-05-24 23:59:59
5,laba06s,2020-05-24 23:59:59
3,project1,2020-05-16 23:59:59


In [6]:
df_min = pd.io.sql.read_sql('''SELECT uid, MIN( 
                    CAST((JulianDay(test.first_commit_ts) - 
                        JulianDay(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 
                                As Integer)) AS min_diff
                    FROM test
                    LEFT JOIN deadlines
                    ON test.labname = deadlines.labs
                    WHERE labname <> 'project1'
                    ''', con)

In [7]:
df_min

,uid,min_diff
0,user_30,-202


In [8]:
df_max = pd.io.sql.read_sql('''SELECT uid, MAX( 
                    CAST((JulianDay(test.first_commit_ts) - 
                        JulianDay(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 
                                As Integer)) AS max_diff
                    FROM test
                    LEFT JOIN deadlines
                    ON test.labname = deadlines.labs
                    WHERE labname <> 'project1'
                    ''', con)

In [9]:
df_max 

,uid,max_diff
0,user_25,-2


In [10]:
df_avg = pd.io.sql.read_sql('''SELECT AVG(
                    CAST((JulianDay(test.first_commit_ts) - 
                        JulianDay(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 
                                As Integer)) AS avg_diff
                    FROM test
                    LEFT JOIN deadlines
                    ON test.labname = deadlines.labs
                    WHERE labname <> 'project1'
                    ''', con)

In [11]:
df_avg

,avg_diff
0,-89.125


In [12]:
views_diff = pd.io.sql.read_sql('''SELECT test.uid, AVG(
                    CAST((JulianDay(test.first_commit_ts) - 
                        JulianDay(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 
                                As Integer)) AS avg_diff, 
                    pageviews
                    FROM test
                    LEFT JOIN deadlines
                    ON test.labname = deadlines.labs
                    LEFT JOIN 
                        (SELECT uid, count(*) as  pageviews
                       FROM pageviews
                       GROUP BY uid) as views
                    ON test.uid = views.uid
                    WHERE labname <> 'project1'
                    GROUP BY test.uid
                    ''', con)

In [13]:
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [14]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [15]:
con.close()